In [ ]:
from endogen.endogen import EndogenousSystem
from endogen.config import GlobalSimConfig, InputModel, ExogenModel, Lags

from dataclasses import asdict
import hydra
from hydra.utils import instantiate

from mlforecast.forecast import MLForecast
from sklearn.linear_model import LinearRegression

# Minimal example without using configuration files

In [ ]:
gc = GlobalSimConfig(input_data = "data/cy_data_static_test.csv",
                     time_var = "year",
                     unit_var = "gwcode",
                     nsim = 10,
                     end = 2100,
                     include_past_n = 30,
                     start = 2022,
                     vars = ['gdppc', 'psecprop', 'population'])

gdppc_model = InputModel(stage = "writing",
           output_var= "gdppc",
           input_vars = ["gdppc_l1", "psecprop_l1", "population_l1"],
           model = MLForecast(models = LinearRegression()),
           lags = [Lags(num_lag = 1, input_vars = ["gdppc", "psecprop", "population"])])

edu_model = InputModel(stage = "writing",
           output_var= "psecprop",
           input_vars = ["gdppc_l1", "psecprop_l1"],
           model = MLForecast(models = LinearRegression()),
           lags = [Lags(num_lag = 1, input_vars = ["gdppc", "psecprop"])])

pop_model = ExogenModel(output_var = "population",
                        exogen_data = "data/pop_test_exog.csv")

s = EndogenousSystem(**asdict(gc))
s.models.add_models([gdppc_model, edu_model, pop_model])
s.create_forecast_container()
s.fit_models()
s.simulate()

## Plot results

In [ ]:
s.plot("gdppc", unit = 475)
s.plot("psecprop", unit = 475)
s.plot("population", unit = 475)
s.plot("population_l1", unit = 475)

## A graph representation of the model system

In [ ]:
s.models.plot()

# A more expansive example using configuration files

Here we use the configuration files found in the "conf" sub-folder. The conf/config.yaml points to .yaml files in conf/global_config and conf/variables. We load these files with hydra. When instantiating, the configuration is made into Python classes defined in endogen/config.py. These are helpers that make it easier to set up our system of endogenous variables.

In [ ]:
with hydra.initialize(version_base = "1.3", config_path="conf"):
    config = hydra.compose(config_name = "config")
    config = instantiate(config, _convert_="all")

In [ ]:
s = EndogenousSystem(**asdict(config.global_config))
s.models.add_models(config.variables)
s.create_forecast_container()

The model scheduler uses the graph representation of our endogenous system, breaks it down into a tranformation step and a forecast step, and determines the sequence calculations must be done. It returns a list of lists that the system then can iterate over. The scheduler is baked into the simulation system, and you do not need to interact with this yourself.

In [ ]:
t0, t1 = s.models._model_schedule
t0, t1

In [ ]:
s.models.plot()

In [ ]:
s.fit_models()

In [ ]:
s.simulate()

In [ ]:
s.plot("gdppc", unit = 475)
s.plot("intensity_level", unit = 475)
s.plot("v2x_libdem", unit = 475)
s.plot("grwth", unit = 475)
s.plot("psecprop", unit = 475)
s.plot("population", unit = 475)
s.plot("rgdp", unit = 475)